<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/Bi-LSTM-GRU-inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, GRU
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
scaler = StandardScaler()
encoder = LabelEncoder()

In [4]:
train = pd.read_csv('/content/drive/MyDrive/train-intervehicle.csv', chunksize=500000)
test = pd.read_csv('/content/drive/MyDrive/test-intervehicle.csv')

In [5]:
model = Sequential()
model.add(Bidirectional(LSTM(100, activation='tanh', recurrent_activation='sigmoid', go_backwards=True, return_sequences=True), input_shape=(len(test.columns)-1, 1)))
model.add(Dense(100, activation='ReLU'))
model.add(GRU(100, activation='tanh', recurrent_activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 70, 200)          81600     
 l)                                                              
                                                                 
 dense (Dense)               (None, 70, 100)           20100     
                                                                 
 gru (GRU)                   (None, 100)               60600     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 162,401
Trainable params: 162,401
Non-trainable params: 0
_________________________________________________________________


In [6]:
for chunk in train:
  train_y = chunk.pop('Label')
  train_x = chunk
  train_x = scaler.fit_transform(train_x)
  train_y = encoder.fit_transform(train_y)
  model.fit(train_x, train_y, batch_size=32)

6161/6161 [==============================] - 66s 11ms/step - loss: 4.5377e-04 - accuracy: 0.9999


In [7]:
test_y = test.pop('Label')
test_x = test

In [8]:
test_x = scaler.transform(test_x)
test_y = encoder.transform(test_y)

In [9]:
pred = model.predict(test_x, verbose=1)
pred_y = np.round(pred).astype(int)

79666/79666 [==============================] - 446s 6ms/step


In [10]:
confusion_matrix(test_y, pred_y)

array([[1255561,     180],
       [      3, 1293544]])

In [11]:
report = classification_report(test_y, pred_y, digits=5)
print(report)

              precision    recall  f1-score   support

           0    1.00000   0.99986   0.99993   1255741
           1    0.99986   1.00000   0.99993   1293547

    accuracy                        0.99993   2549288
   macro avg    0.99993   0.99993   0.99993   2549288
weighted avg    0.99993   0.99993   0.99993   2549288

